In [1]:
import pandas as pd
from func import cal_buoyancy
from sklearn.metrics import mean_squared_error
import glob
import os
import re
import traceback
from scipy.interpolate import interp1d
import requests
import datetime
import metpy.calc as mpcalc
from metpy.units import units
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
from scipy.interpolate import interp1d
from scipy.optimize import bisect
import numpy as np
import pickle
import numpy as np
import math
import pint
from pydoc import help
import xarray as xr
import netCDF4 as nc
import re
import matplotlib.pyplot as plt
from numpy import load
from numpy import asarray
from numpy import save
import pytz
from scipy import stats
from sklearn.linear_model import LinearRegression
from collections import Counter
import pymannkendall as mk
%matplotlib inline

In [2]:
!jupyter nbconvert --to script /home/annierosen16/func.ipynb

[NbConvertApp] Converting notebook /home/annierosen16/func.ipynb to script
[NbConvertApp] Writing 45691 bytes to /home/annierosen16/func.py


In [2]:
# reading in ERA5 specific humidity  

q_path = '/home/annierosen16/data/era5sgp/era5_ml_q_full_sgp'

file_pattern = f"{q_path}/full_q_profile_*"

file_list = glob.glob(file_pattern)

all_q = xr.open_mfdataset(file_list, combine='by_coords', chunks={'time': 24})

all_q = all_q.assign_coords(valid_time=all_q.valid_time - pd.Timedelta(hours=6))

qdf = all_q.sel(valid_time=all_q.valid_time[all_q.valid_time.dt.hour == 12])

qdf['valid_time'].shape, qdf['pressure_level'].data.shape

((2754,), (37,))

In [4]:
# reading in ERA5 geopotential height & temperature

tz_path = '/home/annierosen16/data/era5sgp/era5_ml_t_geopotential_full_sgp'

file_pattern = f"{tz_path}/ERA5_*"

file_list = glob.glob(file_pattern)

all_tz = xr.open_mfdataset(file_list, combine='by_coords', chunks={'time': 24})

# all_tz = all_tz.sel(pressure_level=(all_tz.pressure_level >= 450) & (all_tz.pressure_level <= 1000))

all_tz = all_tz.assign_coords(valid_time=all_tz.valid_time - pd.Timedelta(hours=6))

tzdf = all_tz.sel(valid_time=all_tz.valid_time[all_tz.valid_time.dt.hour == 12])

tzdf['valid_time'].shape, tzdf['pressure_level'].data.shape

((2754,), (37,))

In [5]:
# ERA5 information: surface pressure

sfc_pressure_path = '/data/rong4/Data/ERA5/3hourly/quvw_US/surface_pressure'

all_files = glob.glob(os.path.join(sfc_pressure_path, "era5.surface_pressure.*.nc"))

# Filter files for years 2001-2018 and months 5-9

filtered_files = [
    
    file for file in all_files 
    
    if 2001 <= int(os.path.basename(file).split(".")[2][:4]) <= 2018  
    
    and 5 <= int(os.path.basename(file).split(".")[2][4:]) <= 9       
]

all_sfc_p = xr.open_mfdataset(filtered_files, combine='by_coords')

all_sfc_p = all_sfc_p.sel(latitude=slice(39.0, 30.0), longitude=slice(-105.0, -95.0))

sfcp = all_sfc_p.assign_coords(time=all_sfc_p.time - pd.Timedelta(hours=6))

sfcp = sfcp.sel(time=sfcp.time[sfcp.time.dt.hour == 12])

sfcp['time'].shape

(2754,)

In [6]:
# reading in ERA5 planetary boundary layer height  

pblh_path = '/home/annierosen16/data/era5sgp/era5_sl_pblh_full_sgp'

file_pattern = f"{pblh_path}/pblh_*"

file_list = glob.glob(file_pattern)

all_pblh = xr.open_mfdataset(file_list, combine='by_coords', chunks={'time': 24})

all_pblh = all_pblh.assign_coords(valid_time=all_pblh.valid_time - pd.Timedelta(hours=6))

pblhdf = all_pblh.sel(valid_time=all_pblh.valid_time[all_pblh.valid_time.dt.hour == 12])

pblhdf = pblhdf.where((pblhdf.longitude >= -105) & (pblhdf.longitude <= -95), drop=True)

pblhdf['valid_time'].shape

(2754,)

In [7]:
# loading all necessary information for buoyancy 

g = 9.80665 # m/s^2

# 1 dimensional

times = qdf['valid_time'].data

# 3 dimensional (time, lat, lon)

pblh = pblhdf['blh'].load().data

# 3 dimensional (time, lat, lon)

sp = sfcp['sp'].load().data / 100 # conversion to hPa

# 4 dimensional (time, pressure, lat, lon)

q = qdf['q'].load().data * 1000 # conversion to g/kg

# 4 dimensional (time, pressure - reversed to go from bottom to top -, lat, lon)

t = tzdf['t'].load().data[:, ::-1, :, :] - 273.15 # conversion to Celsius

# 4 dimensional (time, pressure - reversed to go from bottom to top -, lat, lon)

z = tzdf['z'].load().data[:, ::-1, :, :] / g 

lats = tzdf['latitude'].data

lons = tzdf['longitude'].data

In [14]:
# step 1 -> interpolate pressure to altitude

def interpolate_pressure_to_altitude(pres_array, alt_array, surface_pressure):
    
    x = alt_array
    
    y = pres_array

    f = interp1d(x, y, assume_sorted=False, fill_value="extrapolate")

    max_x = np.nanmax(x)
    
    min_x = np.nanmin(x)

    xint = np.arange(min_x, max_x, 20)

    # Interpolate pressure against new altitude intervals
    
    yint = f(xint) * units.hPa
    
    yint = yint.magnitude  # Extract numerical values

    # Compute altitude above surface
    
    mask = yint <= surface_pressure
    
    xnew = xint[mask]
    
    ynew = yint[mask]

    if xnew.size == 0:
        
        return np.array([]), np.array([]), np.nan 

    sfc_alt = xnew[0]  # Define surface altitude before adjusting xnew
    
    xnew_rel = xnew - sfc_alt

    # Apply single mask to keep values within 5000m AGL
    
    valid_mask = xnew_rel <= 4000 # meters AGL
    
    xclean = xnew_rel[valid_mask]
    
    if xclean[-1] != 4000:
        
        print(xclean[-1])
    
    yclean = ynew[valid_mask]

    return yclean, sfc_alt


In [15]:
# for all days (at 12 LST), latitudes & longitudes: interoplate pressure 

num_levels = 201 # equivalent to 4000 meters AGL for every gridpoint

num_times = times.shape[0]

# from bottom to top 

pres_array = tzdf['pressure_level'].data[::-1]

interpolated_pressure = np.full((num_times, num_levels, lats.shape[0], lons.shape[0]), np.nan)

# surface_altitudes captures geopotential height at the surface for all times and gridpoints

surface_altitudes = np.full((num_times, lats.shape[0], lons.shape[0]), np.nan)

for tidx in range(times.shape[0]):
    
#     print(times[tidx])

    for latidx in range(lats.shape[0]):

        for lonidx in range(lons.shape[0]):

            alt_array = z[tidx, :, latidx, lonidx]

            sfc_pres = sp[tidx, latidx, lonidx]

            new_pres, sfc_alt = interpolate_pressure_to_altitude(pres_array, alt_array, sfc_pres)

            interpolated_pressure[tidx, :, latidx, lonidx] = new_pres

            surface_altitudes[tidx, latidx, lonidx] = sfc_alt

In [26]:
# cleaner version

def interpolate_var_to_altitude(variable_to_interpolate, alt_array, sfc_alt):
    
    x = alt_array
    
    y = variable_to_interpolate

    f = interp1d(x, y, assume_sorted=False, fill_value="extrapolate")

    max_x = np.nanmax(x)
    
    min_x = np.nanmin(x)

    xnew = np.arange(min_x, max_x, 20)
    
    ynew = f(xnew)

    # Compute relative altitude
    
    xnew_rel = xnew - sfc_alt
    
    valid_mask = np.logical_and(xnew_rel >= 0, xnew_rel <= 4000)

    yfinal = ynew[valid_mask]
    
    xfinal = xnew_rel[valid_mask]

    return yfinal

In [28]:
# interpolation of specific humidity 

num_levels = 201 # equivalent to 4000 meters AGL for every gridpoint

num_times = times.shape[0]

interpolated_q = np.full((num_times, num_levels, lats.shape[0], lons.shape[0]), np.nan)

for tidx in range(times.shape[0]):
    
    print(times[tidx])

    for latidx in range(lats.shape[0]):

        for lonidx in range(lons.shape[0]):
            
            q_array = q[tidx, :, latidx, lonidx]

            alt_array = z[tidx, :, latidx, lonidx]

            sfc_alt = surface_altitudes[tidx, latidx, lonidx]

            new_q = interpolate_var_to_altitude(q_array, alt_array, sfc_alt)

            interpolated_q[tidx, :, latidx, lonidx] = new_q

2001-05-01T12:00:00.000000000
2001-05-02T12:00:00.000000000
2001-05-03T12:00:00.000000000
2001-05-04T12:00:00.000000000
2001-05-05T12:00:00.000000000
2001-05-06T12:00:00.000000000
2001-05-07T12:00:00.000000000
2001-05-08T12:00:00.000000000
2001-05-09T12:00:00.000000000
2001-05-10T12:00:00.000000000
2001-05-11T12:00:00.000000000
2001-05-12T12:00:00.000000000
2001-05-13T12:00:00.000000000
2001-05-14T12:00:00.000000000
2001-05-15T12:00:00.000000000
2001-05-16T12:00:00.000000000
2001-05-17T12:00:00.000000000
2001-05-18T12:00:00.000000000
2001-05-19T12:00:00.000000000
2001-05-20T12:00:00.000000000
2001-05-21T12:00:00.000000000
2001-05-22T12:00:00.000000000
2001-05-23T12:00:00.000000000
2001-05-24T12:00:00.000000000
2001-05-25T12:00:00.000000000
2001-05-26T12:00:00.000000000
2001-05-27T12:00:00.000000000
2001-05-28T12:00:00.000000000
2001-05-29T12:00:00.000000000
2001-05-30T12:00:00.000000000
2001-05-31T12:00:00.000000000
2001-06-01T12:00:00.000000000
2001-06-02T12:00:00.000000000
2001-06-03

2002-08-30T12:00:00.000000000
2002-08-31T12:00:00.000000000
2002-09-01T12:00:00.000000000
2002-09-02T12:00:00.000000000
2002-09-03T12:00:00.000000000
2002-09-04T12:00:00.000000000
2002-09-05T12:00:00.000000000
2002-09-06T12:00:00.000000000
2002-09-07T12:00:00.000000000
2002-09-08T12:00:00.000000000
2002-09-09T12:00:00.000000000
2002-09-10T12:00:00.000000000
2002-09-11T12:00:00.000000000
2002-09-12T12:00:00.000000000
2002-09-13T12:00:00.000000000
2002-09-14T12:00:00.000000000
2002-09-15T12:00:00.000000000
2002-09-16T12:00:00.000000000
2002-09-17T12:00:00.000000000
2002-09-18T12:00:00.000000000
2002-09-19T12:00:00.000000000
2002-09-20T12:00:00.000000000
2002-09-21T12:00:00.000000000
2002-09-22T12:00:00.000000000
2002-09-23T12:00:00.000000000
2002-09-24T12:00:00.000000000
2002-09-25T12:00:00.000000000
2002-09-26T12:00:00.000000000
2002-09-27T12:00:00.000000000
2002-09-28T12:00:00.000000000
2002-09-29T12:00:00.000000000
2002-09-30T12:00:00.000000000
2003-05-01T12:00:00.000000000
2003-05-02

2004-07-29T12:00:00.000000000
2004-07-30T12:00:00.000000000
2004-07-31T12:00:00.000000000
2004-08-01T12:00:00.000000000
2004-08-02T12:00:00.000000000
2004-08-03T12:00:00.000000000
2004-08-04T12:00:00.000000000
2004-08-05T12:00:00.000000000
2004-08-06T12:00:00.000000000
2004-08-07T12:00:00.000000000
2004-08-08T12:00:00.000000000
2004-08-09T12:00:00.000000000
2004-08-10T12:00:00.000000000
2004-08-11T12:00:00.000000000
2004-08-12T12:00:00.000000000
2004-08-13T12:00:00.000000000
2004-08-14T12:00:00.000000000
2004-08-15T12:00:00.000000000
2004-08-16T12:00:00.000000000
2004-08-17T12:00:00.000000000
2004-08-18T12:00:00.000000000
2004-08-19T12:00:00.000000000
2004-08-20T12:00:00.000000000
2004-08-21T12:00:00.000000000
2004-08-22T12:00:00.000000000
2004-08-23T12:00:00.000000000
2004-08-24T12:00:00.000000000
2004-08-25T12:00:00.000000000
2004-08-26T12:00:00.000000000
2004-08-27T12:00:00.000000000
2004-08-28T12:00:00.000000000
2004-08-29T12:00:00.000000000
2004-08-30T12:00:00.000000000
2004-08-31

2006-06-27T12:00:00.000000000
2006-06-28T12:00:00.000000000
2006-06-29T12:00:00.000000000
2006-06-30T12:00:00.000000000
2006-07-01T12:00:00.000000000
2006-07-02T12:00:00.000000000
2006-07-03T12:00:00.000000000
2006-07-04T12:00:00.000000000
2006-07-05T12:00:00.000000000
2006-07-06T12:00:00.000000000
2006-07-07T12:00:00.000000000
2006-07-08T12:00:00.000000000
2006-07-09T12:00:00.000000000
2006-07-10T12:00:00.000000000
2006-07-11T12:00:00.000000000
2006-07-12T12:00:00.000000000
2006-07-13T12:00:00.000000000
2006-07-14T12:00:00.000000000
2006-07-15T12:00:00.000000000
2006-07-16T12:00:00.000000000
2006-07-17T12:00:00.000000000
2006-07-18T12:00:00.000000000
2006-07-19T12:00:00.000000000
2006-07-20T12:00:00.000000000
2006-07-21T12:00:00.000000000
2006-07-22T12:00:00.000000000
2006-07-23T12:00:00.000000000
2006-07-24T12:00:00.000000000
2006-07-25T12:00:00.000000000
2006-07-26T12:00:00.000000000
2006-07-27T12:00:00.000000000
2006-07-28T12:00:00.000000000
2006-07-29T12:00:00.000000000
2006-07-30

2008-05-26T12:00:00.000000000
2008-05-27T12:00:00.000000000
2008-05-28T12:00:00.000000000
2008-05-29T12:00:00.000000000
2008-05-30T12:00:00.000000000
2008-05-31T12:00:00.000000000
2008-06-01T12:00:00.000000000
2008-06-02T12:00:00.000000000
2008-06-03T12:00:00.000000000
2008-06-04T12:00:00.000000000
2008-06-05T12:00:00.000000000
2008-06-06T12:00:00.000000000
2008-06-07T12:00:00.000000000
2008-06-08T12:00:00.000000000
2008-06-09T12:00:00.000000000
2008-06-10T12:00:00.000000000
2008-06-11T12:00:00.000000000
2008-06-12T12:00:00.000000000
2008-06-13T12:00:00.000000000
2008-06-14T12:00:00.000000000
2008-06-15T12:00:00.000000000
2008-06-16T12:00:00.000000000
2008-06-17T12:00:00.000000000
2008-06-18T12:00:00.000000000
2008-06-19T12:00:00.000000000
2008-06-20T12:00:00.000000000
2008-06-21T12:00:00.000000000
2008-06-22T12:00:00.000000000
2008-06-23T12:00:00.000000000
2008-06-24T12:00:00.000000000
2008-06-25T12:00:00.000000000
2008-06-26T12:00:00.000000000
2008-06-27T12:00:00.000000000
2008-06-28

...

2011-08-23T12:00:00.000000000
2011-08-24T12:00:00.000000000
2011-08-25T12:00:00.000000000
2011-08-26T12:00:00.000000000
2011-08-27T12:00:00.000000000
2011-08-28T12:00:00.000000000
2011-08-29T12:00:00.000000000
2011-08-30T12:00:00.000000000
2011-08-31T12:00:00.000000000
2011-09-01T12:00:00.000000000
2011-09-02T12:00:00.000000000
2011-09-03T12:00:00.000000000
2011-09-04T12:00:00.000000000
2011-09-05T12:00:00.000000000
2011-09-06T12:00:00.000000000
2011-09-07T12:00:00.000000000
2011-09-08T12:00:00.000000000
2011-09-09T12:00:00.000000000
2011-09-10T12:00:00.000000000
2011-09-11T12:00:00.000000000
2011-09-12T12:00:00.000000000
2011-09-13T12:00:00.000000000
2011-09-14T12:00:00.000000000
2011-09-15T12:00:00.000000000
2011-09-16T12:00:00.000000000
2011-09-17T12:00:00.000000000
2011-09-18T12:00:00.000000000
2011-09-19T12:00:00.000000000
2011-09-20T12:00:00.000000000
2011-09-21T12:00:00.000000000
2011-09-22T12:00:00.000000000
2011-09-23T12:00:00.000000000
2011-09-24T12:00:00.000000000
2011-09-25

2013-07-22T12:00:00.000000000
2013-07-23T12:00:00.000000000
2013-07-24T12:00:00.000000000
2013-07-25T12:00:00.000000000
2013-07-26T12:00:00.000000000
2013-07-27T12:00:00.000000000
2013-07-28T12:00:00.000000000
2013-07-29T12:00:00.000000000
2013-07-30T12:00:00.000000000
2013-07-31T12:00:00.000000000
2013-08-01T12:00:00.000000000
2013-08-02T12:00:00.000000000
2013-08-03T12:00:00.000000000
2013-08-04T12:00:00.000000000
2013-08-05T12:00:00.000000000
2013-08-06T12:00:00.000000000
2013-08-07T12:00:00.000000000
2013-08-08T12:00:00.000000000
2013-08-09T12:00:00.000000000
2013-08-10T12:00:00.000000000
2013-08-11T12:00:00.000000000
2013-08-12T12:00:00.000000000
2013-08-13T12:00:00.000000000
2013-08-14T12:00:00.000000000
2013-08-15T12:00:00.000000000
2013-08-16T12:00:00.000000000
2013-08-17T12:00:00.000000000
2013-08-18T12:00:00.000000000
2013-08-19T12:00:00.000000000
2013-08-20T12:00:00.000000000
2013-08-21T12:00:00.000000000
2013-08-22T12:00:00.000000000
2013-08-23T12:00:00.000000000
2013-08-24

2015-06-20T12:00:00.000000000
2015-06-21T12:00:00.000000000
2015-06-22T12:00:00.000000000
2015-06-23T12:00:00.000000000
2015-06-24T12:00:00.000000000
2015-06-25T12:00:00.000000000
2015-06-26T12:00:00.000000000
2015-06-27T12:00:00.000000000
2015-06-28T12:00:00.000000000
2015-06-29T12:00:00.000000000
2015-06-30T12:00:00.000000000
2015-07-01T12:00:00.000000000
2015-07-02T12:00:00.000000000
2015-07-03T12:00:00.000000000
2015-07-04T12:00:00.000000000
2015-07-05T12:00:00.000000000
2015-07-06T12:00:00.000000000
2015-07-07T12:00:00.000000000
2015-07-08T12:00:00.000000000
2015-07-09T12:00:00.000000000
2015-07-10T12:00:00.000000000
2015-07-11T12:00:00.000000000
2015-07-12T12:00:00.000000000
2015-07-13T12:00:00.000000000
2015-07-14T12:00:00.000000000
2015-07-15T12:00:00.000000000
2015-07-16T12:00:00.000000000
2015-07-17T12:00:00.000000000
2015-07-18T12:00:00.000000000
2015-07-19T12:00:00.000000000
2015-07-20T12:00:00.000000000
2015-07-21T12:00:00.000000000
2015-07-22T12:00:00.000000000
2015-07-23

2017-05-19T12:00:00.000000000
2017-05-20T12:00:00.000000000
2017-05-21T12:00:00.000000000
2017-05-22T12:00:00.000000000
2017-05-23T12:00:00.000000000
2017-05-24T12:00:00.000000000
2017-05-25T12:00:00.000000000
2017-05-26T12:00:00.000000000
2017-05-27T12:00:00.000000000
2017-05-28T12:00:00.000000000
2017-05-29T12:00:00.000000000
2017-05-30T12:00:00.000000000
2017-05-31T12:00:00.000000000
2017-06-01T12:00:00.000000000
2017-06-02T12:00:00.000000000
2017-06-03T12:00:00.000000000
2017-06-04T12:00:00.000000000
2017-06-05T12:00:00.000000000
2017-06-06T12:00:00.000000000
2017-06-07T12:00:00.000000000
2017-06-08T12:00:00.000000000
2017-06-09T12:00:00.000000000
2017-06-10T12:00:00.000000000
2017-06-11T12:00:00.000000000
2017-06-12T12:00:00.000000000
2017-06-13T12:00:00.000000000
2017-06-14T12:00:00.000000000
2017-06-15T12:00:00.000000000
2017-06-16T12:00:00.000000000
2017-06-17T12:00:00.000000000
2017-06-18T12:00:00.000000000
2017-06-19T12:00:00.000000000
2017-06-20T12:00:00.000000000
2017-06-21

2018-09-17T12:00:00.000000000
2018-09-18T12:00:00.000000000
2018-09-19T12:00:00.000000000
2018-09-20T12:00:00.000000000
2018-09-21T12:00:00.000000000
2018-09-22T12:00:00.000000000
2018-09-23T12:00:00.000000000
2018-09-24T12:00:00.000000000
2018-09-25T12:00:00.000000000
2018-09-26T12:00:00.000000000
2018-09-27T12:00:00.000000000
2018-09-28T12:00:00.000000000
2018-09-29T12:00:00.000000000
2018-09-30T12:00:00.000000000


In [32]:
# interpolation of temperature 

num_levels = 201 # equivalent to 4000 meters AGL for every gridpoint

num_times = times.shape[0]

interpolated_t = np.full((num_times, num_levels, lats.shape[0], lons.shape[0]), np.nan)

for tidx in range(times.shape[0]):
    
    print(times[tidx])

    for latidx in range(lats.shape[0]):

        for lonidx in range(lons.shape[0]):
            
            t_array = t[tidx, :, latidx, lonidx]

            alt_array = z[tidx, :, latidx, lonidx]

            sfc_alt = surface_altitudes[tidx, latidx, lonidx]

            new_t = interpolate_var_to_altitude(t_array, alt_array, sfc_alt)

            interpolated_t[tidx, :, latidx, lonidx] = new_t

2001-05-01T12:00:00.000000000
2001-05-02T12:00:00.000000000
2001-05-03T12:00:00.000000000
2001-05-04T12:00:00.000000000
2001-05-05T12:00:00.000000000
2001-05-06T12:00:00.000000000
2001-05-07T12:00:00.000000000
2001-05-08T12:00:00.000000000
2001-05-09T12:00:00.000000000
2001-05-10T12:00:00.000000000
2001-05-11T12:00:00.000000000
2001-05-12T12:00:00.000000000
2001-05-13T12:00:00.000000000
2001-05-14T12:00:00.000000000
2001-05-15T12:00:00.000000000
2001-05-16T12:00:00.000000000
2001-05-17T12:00:00.000000000
2001-05-18T12:00:00.000000000
2001-05-19T12:00:00.000000000
2001-05-20T12:00:00.000000000
2001-05-21T12:00:00.000000000
2001-05-22T12:00:00.000000000
2001-05-23T12:00:00.000000000
2001-05-24T12:00:00.000000000
2001-05-25T12:00:00.000000000
2001-05-26T12:00:00.000000000
2001-05-27T12:00:00.000000000
2001-05-28T12:00:00.000000000
2001-05-29T12:00:00.000000000
2001-05-30T12:00:00.000000000
2001-05-31T12:00:00.000000000
2001-06-01T12:00:00.000000000
2001-06-02T12:00:00.000000000
2001-06-03

2002-08-30T12:00:00.000000000
2002-08-31T12:00:00.000000000
2002-09-01T12:00:00.000000000
2002-09-02T12:00:00.000000000
2002-09-03T12:00:00.000000000
2002-09-04T12:00:00.000000000
2002-09-05T12:00:00.000000000
2002-09-06T12:00:00.000000000
2002-09-07T12:00:00.000000000
2002-09-08T12:00:00.000000000
2002-09-09T12:00:00.000000000
2002-09-10T12:00:00.000000000
2002-09-11T12:00:00.000000000
2002-09-12T12:00:00.000000000
2002-09-13T12:00:00.000000000
2002-09-14T12:00:00.000000000
2002-09-15T12:00:00.000000000
2002-09-16T12:00:00.000000000
2002-09-17T12:00:00.000000000
2002-09-18T12:00:00.000000000
2002-09-19T12:00:00.000000000
2002-09-20T12:00:00.000000000
2002-09-21T12:00:00.000000000
2002-09-22T12:00:00.000000000
2002-09-23T12:00:00.000000000
2002-09-24T12:00:00.000000000
2002-09-25T12:00:00.000000000
2002-09-26T12:00:00.000000000
2002-09-27T12:00:00.000000000
2002-09-28T12:00:00.000000000
2002-09-29T12:00:00.000000000
2002-09-30T12:00:00.000000000
2003-05-01T12:00:00.000000000
2003-05-02

2004-07-29T12:00:00.000000000
2004-07-30T12:00:00.000000000
2004-07-31T12:00:00.000000000
2004-08-01T12:00:00.000000000
2004-08-02T12:00:00.000000000
2004-08-03T12:00:00.000000000
2004-08-04T12:00:00.000000000
2004-08-05T12:00:00.000000000
2004-08-06T12:00:00.000000000
2004-08-07T12:00:00.000000000
2004-08-08T12:00:00.000000000
2004-08-09T12:00:00.000000000
2004-08-10T12:00:00.000000000
2004-08-11T12:00:00.000000000
2004-08-12T12:00:00.000000000
2004-08-13T12:00:00.000000000
2004-08-14T12:00:00.000000000
2004-08-15T12:00:00.000000000
2004-08-16T12:00:00.000000000
2004-08-17T12:00:00.000000000
2004-08-18T12:00:00.000000000
2004-08-19T12:00:00.000000000
2004-08-20T12:00:00.000000000
2004-08-21T12:00:00.000000000
2004-08-22T12:00:00.000000000
2004-08-23T12:00:00.000000000
2004-08-24T12:00:00.000000000
2004-08-25T12:00:00.000000000
2004-08-26T12:00:00.000000000
2004-08-27T12:00:00.000000000
2004-08-28T12:00:00.000000000
2004-08-29T12:00:00.000000000
2004-08-30T12:00:00.000000000
2004-08-31

2006-06-27T12:00:00.000000000
2006-06-28T12:00:00.000000000
2006-06-29T12:00:00.000000000
2006-06-30T12:00:00.000000000
2006-07-01T12:00:00.000000000
2006-07-02T12:00:00.000000000
2006-07-03T12:00:00.000000000
2006-07-04T12:00:00.000000000
2006-07-05T12:00:00.000000000
2006-07-06T12:00:00.000000000
2006-07-07T12:00:00.000000000
2006-07-08T12:00:00.000000000
2006-07-09T12:00:00.000000000
2006-07-10T12:00:00.000000000
2006-07-11T12:00:00.000000000
2006-07-12T12:00:00.000000000
2006-07-13T12:00:00.000000000
2006-07-14T12:00:00.000000000
2006-07-15T12:00:00.000000000
2006-07-16T12:00:00.000000000
2006-07-17T12:00:00.000000000
2006-07-18T12:00:00.000000000
2006-07-19T12:00:00.000000000
2006-07-20T12:00:00.000000000
2006-07-21T12:00:00.000000000
2006-07-22T12:00:00.000000000
2006-07-23T12:00:00.000000000
2006-07-24T12:00:00.000000000
2006-07-25T12:00:00.000000000
2006-07-26T12:00:00.000000000
2006-07-27T12:00:00.000000000
2006-07-28T12:00:00.000000000
2006-07-29T12:00:00.000000000
2006-07-30

2008-05-26T12:00:00.000000000
2008-05-27T12:00:00.000000000
2008-05-28T12:00:00.000000000
2008-05-29T12:00:00.000000000
2008-05-30T12:00:00.000000000
2008-05-31T12:00:00.000000000
2008-06-01T12:00:00.000000000
2008-06-02T12:00:00.000000000
2008-06-03T12:00:00.000000000
2008-06-04T12:00:00.000000000
2008-06-05T12:00:00.000000000
2008-06-06T12:00:00.000000000
2008-06-07T12:00:00.000000000
2008-06-08T12:00:00.000000000
2008-06-09T12:00:00.000000000
2008-06-10T12:00:00.000000000
2008-06-11T12:00:00.000000000
2008-06-12T12:00:00.000000000
2008-06-13T12:00:00.000000000
2008-06-14T12:00:00.000000000
2008-06-15T12:00:00.000000000
2008-06-16T12:00:00.000000000
2008-06-17T12:00:00.000000000
2008-06-18T12:00:00.000000000
2008-06-19T12:00:00.000000000
2008-06-20T12:00:00.000000000
2008-06-21T12:00:00.000000000
2008-06-22T12:00:00.000000000
2008-06-23T12:00:00.000000000
2008-06-24T12:00:00.000000000
2008-06-25T12:00:00.000000000
2008-06-26T12:00:00.000000000
2008-06-27T12:00:00.000000000
2008-06-28

...

2011-08-23T12:00:00.000000000
2011-08-24T12:00:00.000000000
2011-08-25T12:00:00.000000000
2011-08-26T12:00:00.000000000
2011-08-27T12:00:00.000000000
2011-08-28T12:00:00.000000000
2011-08-29T12:00:00.000000000
2011-08-30T12:00:00.000000000
2011-08-31T12:00:00.000000000
2011-09-01T12:00:00.000000000
2011-09-02T12:00:00.000000000
2011-09-03T12:00:00.000000000
2011-09-04T12:00:00.000000000
2011-09-05T12:00:00.000000000
2011-09-06T12:00:00.000000000
2011-09-07T12:00:00.000000000
2011-09-08T12:00:00.000000000
2011-09-09T12:00:00.000000000
2011-09-10T12:00:00.000000000
2011-09-11T12:00:00.000000000
2011-09-12T12:00:00.000000000
2011-09-13T12:00:00.000000000
2011-09-14T12:00:00.000000000
2011-09-15T12:00:00.000000000
2011-09-16T12:00:00.000000000
2011-09-17T12:00:00.000000000
2011-09-18T12:00:00.000000000
2011-09-19T12:00:00.000000000
2011-09-20T12:00:00.000000000
2011-09-21T12:00:00.000000000
2011-09-22T12:00:00.000000000
2011-09-23T12:00:00.000000000
2011-09-24T12:00:00.000000000
2011-09-25

2013-07-22T12:00:00.000000000
2013-07-23T12:00:00.000000000
2013-07-24T12:00:00.000000000
2013-07-25T12:00:00.000000000
2013-07-26T12:00:00.000000000
2013-07-27T12:00:00.000000000
2013-07-28T12:00:00.000000000
2013-07-29T12:00:00.000000000
2013-07-30T12:00:00.000000000
2013-07-31T12:00:00.000000000
2013-08-01T12:00:00.000000000
2013-08-02T12:00:00.000000000
2013-08-03T12:00:00.000000000
2013-08-04T12:00:00.000000000
2013-08-05T12:00:00.000000000
2013-08-06T12:00:00.000000000
2013-08-07T12:00:00.000000000
2013-08-08T12:00:00.000000000
2013-08-09T12:00:00.000000000
2013-08-10T12:00:00.000000000
2013-08-11T12:00:00.000000000
2013-08-12T12:00:00.000000000
2013-08-13T12:00:00.000000000
2013-08-14T12:00:00.000000000
2013-08-15T12:00:00.000000000
2013-08-16T12:00:00.000000000
2013-08-17T12:00:00.000000000
2013-08-18T12:00:00.000000000
2013-08-19T12:00:00.000000000
2013-08-20T12:00:00.000000000
2013-08-21T12:00:00.000000000
2013-08-22T12:00:00.000000000
2013-08-23T12:00:00.000000000
2013-08-24

2015-06-20T12:00:00.000000000
2015-06-21T12:00:00.000000000
2015-06-22T12:00:00.000000000
2015-06-23T12:00:00.000000000
2015-06-24T12:00:00.000000000
2015-06-25T12:00:00.000000000
2015-06-26T12:00:00.000000000
2015-06-27T12:00:00.000000000
2015-06-28T12:00:00.000000000
2015-06-29T12:00:00.000000000
2015-06-30T12:00:00.000000000
2015-07-01T12:00:00.000000000
2015-07-02T12:00:00.000000000
2015-07-03T12:00:00.000000000
2015-07-04T12:00:00.000000000
2015-07-05T12:00:00.000000000
2015-07-06T12:00:00.000000000
2015-07-07T12:00:00.000000000
2015-07-08T12:00:00.000000000
2015-07-09T12:00:00.000000000
2015-07-10T12:00:00.000000000
2015-07-11T12:00:00.000000000
2015-07-12T12:00:00.000000000
2015-07-13T12:00:00.000000000
2015-07-14T12:00:00.000000000
2015-07-15T12:00:00.000000000
2015-07-16T12:00:00.000000000
2015-07-17T12:00:00.000000000
2015-07-18T12:00:00.000000000
2015-07-19T12:00:00.000000000
2015-07-20T12:00:00.000000000
2015-07-21T12:00:00.000000000
2015-07-22T12:00:00.000000000
2015-07-23

2017-05-19T12:00:00.000000000
2017-05-20T12:00:00.000000000
2017-05-21T12:00:00.000000000
2017-05-22T12:00:00.000000000
2017-05-23T12:00:00.000000000
2017-05-24T12:00:00.000000000
2017-05-25T12:00:00.000000000
2017-05-26T12:00:00.000000000
2017-05-27T12:00:00.000000000
2017-05-28T12:00:00.000000000
2017-05-29T12:00:00.000000000
2017-05-30T12:00:00.000000000
2017-05-31T12:00:00.000000000
2017-06-01T12:00:00.000000000
2017-06-02T12:00:00.000000000
2017-06-03T12:00:00.000000000
2017-06-04T12:00:00.000000000
2017-06-05T12:00:00.000000000
2017-06-06T12:00:00.000000000
2017-06-07T12:00:00.000000000
2017-06-08T12:00:00.000000000
2017-06-09T12:00:00.000000000
2017-06-10T12:00:00.000000000
2017-06-11T12:00:00.000000000
2017-06-12T12:00:00.000000000
2017-06-13T12:00:00.000000000
2017-06-14T12:00:00.000000000
2017-06-15T12:00:00.000000000
2017-06-16T12:00:00.000000000
2017-06-17T12:00:00.000000000
2017-06-18T12:00:00.000000000
2017-06-19T12:00:00.000000000
2017-06-20T12:00:00.000000000
2017-06-21

2018-09-17T12:00:00.000000000
2018-09-18T12:00:00.000000000
2018-09-19T12:00:00.000000000
2018-09-20T12:00:00.000000000
2018-09-21T12:00:00.000000000
2018-09-22T12:00:00.000000000
2018-09-23T12:00:00.000000000
2018-09-24T12:00:00.000000000
2018-09-25T12:00:00.000000000
2018-09-26T12:00:00.000000000
2018-09-27T12:00:00.000000000
2018-09-28T12:00:00.000000000
2018-09-29T12:00:00.000000000
2018-09-30T12:00:00.000000000


In [35]:
# creation of q_fit profiles -> for every gridpoint and everyday, calculate the 0-1km ("mixed layer") average q

# the collection of all these average qs across time for each gridpoint is the xinput to the regression

# then, for each height, collect all q values for all times -> this is your y_target

# the lr model is trained on x input and y input and returns a prediction for q values at that height

# note that q_fit is equivalent to original q profiles below 2km

interpolated_altitudes = np.arange(0, 4000 + 20, 20)  

n_times = interpolated_q.shape[0]  

n_altitudes = len(interpolated_altitudes) 

n_lats = lats.shape[0]  

n_lons = lons.shape[0]  

interpolated_qfit = np.full((n_times, n_altitudes, n_lats, n_lons), np.nan, dtype=np.float32)

for latidx in range(n_lats):
    
    for lonidx in range(n_lons):
        
        print(lats[latidx], lons[lonidx])
        
        # extracts all q profiles for a given gridpoint across time series 
        q_array = interpolated_q[:, :, latidx, lonidx]  # shape (time, levels)
        
        # define indices for bottom 1km and 2km
        
        bottom_1km_idx = interpolated_altitudes <= 1000
        
        bottom_2km_idx = interpolated_altitudes <= 2000
        
        target_idx = (interpolated_altitudes > 2000) & (interpolated_altitudes <= 4000)  # heights to predict

        # compute near-surface q (mean of bottom 1km for every profile in time series)
        
        near_surface_q = np.mean(q_array[:, bottom_1km_idx], axis=1).reshape(-1, 1) # shape = (time)

        lr = LinearRegression()

        # copy q_array and retain observed values up to 2km
        
        q_fit = q_array.copy()
        
        q_fit[:, ~bottom_2km_idx] = np.nan  # Keep observed q below 2km

        # loop over target heights (above 2km) and perform regression
        
        for j, idx in enumerate(np.where(target_idx)[0]):
            
            y_target = q_array[:, idx].reshape(-1, 1)  # Shape (2754, 1)
            
            # train regression model
            
            lr.fit(near_surface_q, y_target)
            
            # Predict q_fit values for this height and assign them to the large array
            
            q_fit[:, idx] = lr.predict(near_surface_q).flatten()

        # append the q_fit for this lat, lon position into the big array
        
        interpolated_qfit[:, :, latidx, lonidx] = q_fit

print(interpolated_qfit.shape)


39.0 -105.0
39.0 -104.75
39.0 -104.5
39.0 -104.25
39.0 -104.0
39.0 -103.75
39.0 -103.5
39.0 -103.25
39.0 -103.0
39.0 -102.75
39.0 -102.5
39.0 -102.25
39.0 -102.0
39.0 -101.75
39.0 -101.5
39.0 -101.25
39.0 -101.0
39.0 -100.75
39.0 -100.5
39.0 -100.25
39.0 -100.0
39.0 -99.75
39.0 -99.5
39.0 -99.25
39.0 -99.0
39.0 -98.75
39.0 -98.5
39.0 -98.25
39.0 -98.0
39.0 -97.75
39.0 -97.5
39.0 -97.25
39.0 -97.0
39.0 -96.75
39.0 -96.5
39.0 -96.25
39.0 -96.0
39.0 -95.75
39.0 -95.5
39.0 -95.25
39.0 -95.0
38.75 -105.0
38.75 -104.75
38.75 -104.5
38.75 -104.25
38.75 -104.0
38.75 -103.75
38.75 -103.5
38.75 -103.25
38.75 -103.0
38.75 -102.75
38.75 -102.5
38.75 -102.25
38.75 -102.0
38.75 -101.75
38.75 -101.5
38.75 -101.25
38.75 -101.0
38.75 -100.75
38.75 -100.5
38.75 -100.25
38.75 -100.0
38.75 -99.75
38.75 -99.5
38.75 -99.25
38.75 -99.0
38.75 -98.75
38.75 -98.5
38.75 -98.25
38.75 -98.0
38.75 -97.75
38.75 -97.5
38.75 -97.25
38.75 -97.0
38.75 -96.75
38.75 -96.5
38.75 -96.25
38.75 -96.0
38.75 -95.75
38.75 -95.5


35.0 -104.75
35.0 -104.5
35.0 -104.25
35.0 -104.0
35.0 -103.75
35.0 -103.5
35.0 -103.25
35.0 -103.0
35.0 -102.75
35.0 -102.5
35.0 -102.25
35.0 -102.0
35.0 -101.75
35.0 -101.5
35.0 -101.25
35.0 -101.0
35.0 -100.75
35.0 -100.5
35.0 -100.25
35.0 -100.0
35.0 -99.75
35.0 -99.5
35.0 -99.25
35.0 -99.0
35.0 -98.75
35.0 -98.5
35.0 -98.25
35.0 -98.0
35.0 -97.75
35.0 -97.5
35.0 -97.25
35.0 -97.0
35.0 -96.75
35.0 -96.5
35.0 -96.25
35.0 -96.0
35.0 -95.75
35.0 -95.5
35.0 -95.25
35.0 -95.0
34.75 -105.0
34.75 -104.75
34.75 -104.5
34.75 -104.25
34.75 -104.0
34.75 -103.75
34.75 -103.5
34.75 -103.25
34.75 -103.0
34.75 -102.75
34.75 -102.5
34.75 -102.25
34.75 -102.0
34.75 -101.75
34.75 -101.5
34.75 -101.25
34.75 -101.0
34.75 -100.75
34.75 -100.5
34.75 -100.25
34.75 -100.0
34.75 -99.75
34.75 -99.5
34.75 -99.25
34.75 -99.0
34.75 -98.75
34.75 -98.5
34.75 -98.25
34.75 -98.0
34.75 -97.75
34.75 -97.5
34.75 -97.25
34.75 -97.0
34.75 -96.75
34.75 -96.5
34.75 -96.25
34.75 -96.0
34.75 -95.75
34.75 -95.5
34.75 -95.25

31.0 -104.5
31.0 -104.25
31.0 -104.0
31.0 -103.75
31.0 -103.5
31.0 -103.25
31.0 -103.0
31.0 -102.75
31.0 -102.5
31.0 -102.25
31.0 -102.0
31.0 -101.75
31.0 -101.5
31.0 -101.25
31.0 -101.0
31.0 -100.75
31.0 -100.5
31.0 -100.25
31.0 -100.0
31.0 -99.75
31.0 -99.5
31.0 -99.25
31.0 -99.0
31.0 -98.75
31.0 -98.5
31.0 -98.25
31.0 -98.0
31.0 -97.75
31.0 -97.5
31.0 -97.25
31.0 -97.0
31.0 -96.75
31.0 -96.5
31.0 -96.25
31.0 -96.0
31.0 -95.75
31.0 -95.5
31.0 -95.25
31.0 -95.0
30.75 -105.0
30.75 -104.75
30.75 -104.5
30.75 -104.25
30.75 -104.0
30.75 -103.75
30.75 -103.5
30.75 -103.25
30.75 -103.0
30.75 -102.75
30.75 -102.5
30.75 -102.25
30.75 -102.0
30.75 -101.75
30.75 -101.5
30.75 -101.25
30.75 -101.0
30.75 -100.75
30.75 -100.5
30.75 -100.25
30.75 -100.0
30.75 -99.75
30.75 -99.5
30.75 -99.25
30.75 -99.0
30.75 -98.75
30.75 -98.5
30.75 -98.25
30.75 -98.0
30.75 -97.75
30.75 -97.5
30.75 -97.25
30.75 -97.0
30.75 -96.75
30.75 -96.5
30.75 -96.25
30.75 -96.0
30.75 -95.75
30.75 -95.5
30.75 -95.25
30.75 -95.0


In [39]:
# Step 1: Define the backup directory in your home folder

backup_dir = os.path.expanduser("~/backup_data")  # Expands ~ to full home directory path

os.makedirs(backup_dir, exist_ok=True)  # create the directory if it doesn't exist

# Step 3: Save multiple arrays in a compressed file
file_path = os.path.join(backup_dir, "backup_arrays_for_buoyancy.npz")  # Full path to save file

np.savez_compressed(file_path, array1=interpolated_pressure, array2=interpolated_q, array3=interpolated_t,
                   array_4=interpolated_qfit)

print(f"Arrays saved to {file_path}")

# Step 4: Verify that the file exists
print("Saved files:", os.listdir(backup_dir))


Arrays saved to /home/annierosen16/backup_data/backup_arrays_for_buoyancy.npz
Saved files: ['backup_arrays_for_buoyancy.npz']


In [18]:
backup_dir = os.path.expanduser("~/backup_data")

os.makedirs(backup_dir, exist_ok=True)  # Ensure the directory exists

file_path = os.path.join(backup_dir, "backup_metadata_for_buoyancy.npz")

np.savez(file_path, 
         times=times, 
         lats=lats, 
         lons=lons, 
        )

print(f"Backup saved at {file_path}")

Backup saved at /home/annierosen16/backup_data/backup_metadata_for_buoyancy.npz


In [1]:
# Step 1: Define the backup directory in your home folder
backup_dir = os.path.expanduser("~/backup_data")  # Expands ~ to full home directory path

os.makedirs(backup_dir, exist_ok=True)  # create the directory if it doesn't exist

# Step 3: Save multiple arrays in a compressed file
file_path = os.path.join(backup_dir, "backup_arrays_for_buoyancy.npz")  # Full path to save file

# Step 5: Load back the saved arrays
data = np.load(file_path)  # Load the .npz file

interpolated_pressure = data['array1']

print("interpolated_pressure loaded")

interpolated_q = data['array2']

print("interpolated_pressure loaded")

interpolated_t = data['array3']

print("interpolated_t loaded")

interpolated_qfit = data['array_4']

print("interpolated_qfit loaded")

print("")

print("Arrays successfully loaded!")

NameError: name 'os' is not defined

## complete buoyancy script

In [8]:
# backup_dir = os.path.expanduser("~/backup_data")  # Expands ~ to full home directory path

# os.makedirs(backup_dir, exist_ok=True)  # create the directory if it doesn't exist

# file_path = os.path.join(backup_dir, "backup_arrays_for_buoyancy.npz")  # Full path to save file

# # Step 5: Load back the saved arrays
# data = np.load(file_path)  # Load the .npz file

# interpolated_pressure = data['array1']

# print("interpolated_pressure loaded")

# interpolated_q = data['array2']

# print("interpolated_pressure loaded")

# interpolated_t = data['array3']

# print("interpolated_t loaded")

# interpolated_qfit = data['array_4']

# print("interpolated_qfit loaded")

# print("")

# print("Arrays successfully loaded!")

# %run /home/annierosen16/func.ipynb

# all_blt = np.full((times.shape[0], lats.size, lons.size), np.nan)  # Shape (times, lats, lons)

# all_blc = np.full((times.shape[0], lats.size, lons.size), np.nan)  # Shape (times, lats, lons)

# alt = interpolated_altitudes

# g = 9.80665  # m/s^2

# # Loop over all latitudes and longitudes
# for latidx in range(lats.size):
    
#     for lonidx in range(lons.size):
        
#         for tidx in range(times.shape[0]):
            
#             p = interpolated_pressure[tidx, :, latidx, lonidx]
            
#             t = interpolated_t[tidx, :, latidx, lonidx]
            
#             qlc = interpolated_qfit[tidx, :, latidx, lonidx]
            
#             qr = interpolated_q[tidx, :, latidx, lonidx]

#             # Calculate buoyancy
#             b_qr = cal_buoyancy(t_profile=t, h_profile=qr, p_profile=p, alt_profile=alt)
            
#             b_qlc = cal_buoyancy(t_profile=t, h_profile=qlc, p_profile=p, alt_profile=alt)

#             # Convert to Kelvin and calculate y_qr, y_qlc
#             pprof_qr = (b_qr.tp) * units.degK
            
#             pprof_qlc = (b_qlc.tp) * units.degK

#             t = t * units.degC
            
#             t = t.to(units.degK)

#             y_qr = pprof_qr - t
#             y_qlc = pprof_qlc - t

#             # Pressure levels for masks
#             pres_100 = p[0] - 100
            
#             pres_300 = p[0] - 300
            
#             pres_mask = (p >= pres_300) & (p <= pres_100)
            
#             altmask = (alt >= 2000) & (alt <= 4000)

#             x_clipped = p[altmask]
            
#             y_clipped_qr = y_qr[altmask]
            
#             y_clipped_qlc = y_qlc[altmask]

#             # Compute blc, blt
#             br = (mpconsts.Rd * (np.trapz(y_clipped_qr[::-1], np.log(x_clipped[::-1])))).to(units('J/kg'))
            
#             blc = (mpconsts.Rd * (np.trapz(y_clipped_qlc[::-1], np.log(x_clipped[::-1])))).to(units('J/kg'))

#             blt = br - blc

#             all_blt[tidx, latidx, lonidx] = blt.magnitude
            
#             all_blc[tidx, latidx, lonidx] = blc.magnitude

#             print(f"Time index {tidx}, Latitude index {latidx}, Longitude index {lonidx}: blt={blt.magnitude}, blc={blc.magnitude}")



In [ ]:
with open('all_blc.pkl', 'wb') as file:

    pickle.dump(all_blc, file)

with open('all_blt.pkl', 'wb') as file:

    pickle.dump(all_blt, file)   

In [8]:
# testing on one gridpoint

interpolated_altitudes = np.arange(0, 4000 + 20, 20)  

%run /home/annierosen16/func.ipynb

all_blt = []

all_blc = []

g = 9.80665 # m/s^2

# latidx = np.where(lats == 36.5)[0][0]

# lonidx = np.where(lons == -97.5)[0][0]

latidx = 0

lonidx = 0

for tidx in range(times.shape[0]):

    alt = interpolated_altitudes

    p = interpolated_pressure[tidx, :, latidx, lonidx]

    t = interpolated_t[tidx, :, latidx, lonidx]

    qlc = interpolated_qfit[tidx, :, latidx, lonidx]

    qr = interpolated_q[tidx, :, latidx, lonidx]

    b_qr = cal_buoyancy(t_profile = t,         
                            h_profile = qr,         
                            p_profile = p,        
                            alt_profile = alt,
                            )

    b_qlc = cal_buoyancy(t_profile = t,         
                        h_profile = qlc,         
                        p_profile = p,        
                        alt_profile = alt,
                        )

    pprof_qr = (b_qr.tp)*units.degK

    pprof_qlc = (b_qlc.tp)*units.degK

    t = t*units.degC

    t = t.to(units.degK)

    y_qr = pprof_qr - t

    y_qlc = pprof_qlc - t

    pres_100 = p[0]-100

    pres_300 = p[0]-300

    pres_mask = (p >= pres_300) & (p <= pres_100)

    altmask = (alt >= 2000) & (alt <= 4000)

    x_clipped = p[altmask]

    y_clipped_qr = y_qr[altmask]

    y_clipped_qlc = y_qlc[altmask]

    br = (mpconsts.Rd*(np.trapz(y_clipped_qr[::-1], np.log(x_clipped[::-1])))).to(units('J/kg'))

    blc = (mpconsts.Rd*(np.trapz(y_clipped_qlc[::-1], np.log(x_clipped[::-1])))).to(units('J/kg'))

    blt = br - blc

    all_blt.append(blt.magnitude)
    
    all_blc.append(blc.magnitude)

    print(tidx, blt.magnitude, blc.magnitude)

0 -2.9329206702356885 173.4385353021287
1 -3.013063205755401 -166.39931003575225
2 -0.24029919453275284 -98.69692776067645
3 2.7102621789555137 -26.152886595758826
4 1.7936303619939764 89.79457262397922
5 -6.255551565621488 326.48269214873636
6 -4.321669594232475 148.05757368286305
7 -2.417860887235406 235.19201504623376
8 -3.641815322901323 309.41612185502953
9 4.6228416470349885 323.91605822598984
10 -9.579015705044583 210.47301774675032
11 -2.129399845671486 222.25798746163716
12 0.39991495534638943 282.1451292395389
13 1.655969676461865 229.16119341175593
14 -4.051133272537527 93.74937871221539
15 5.368378986077346 232.02507564158876
16 1.6751206148592814 222.38282892920915
17 -4.920088431682558 173.38897792562878
18 1.8747886991704945 168.98209683114695
19 -4.285473050660357 294.3374691812661
20 -3.6283507125761503 405.86216161680386
21 -1.958765695677073 268.2069295255413


KeyboardInterrupt: 